In [ ]:
%matplotlib inline
import os                                               # to set current working directory 
import sys                                              # supress output to screen for interactive variogram modeling
import io
import numpy as np                                      # arrays and matrix math
import pandas as pd                                     # DataFrames
import matplotlib.pyplot as plt                         # plotting
from sklearn.model_selection import train_test_split    # train and test split
from sklearn.metrics import mean_squared_error          # model error calculation
from sklearn.preprocessing import StandardScaler        # standardize data
import scipy                                            # kernel density estimator for PDF plot
from matplotlib.pyplot import cm                        # color maps


import tensorflow as tf                                 # build deep learning models
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam, SGD, RMSprop, Adadelta


from ipywidgets import interactive                      # widgets and interactivity
from ipywidgets import widgets                            
from ipywidgets import Layout
from ipywidgets import Label
from ipywidgets import VBox, HBox
import warnings
warnings.filterwarnings('ignore')                       # supress warnings

In [ ]:
activation_function = ['linear', 'ReLU', 'Sigmoid', 'Softmax', 'Leaky ReLU', 'GELU']
reduce_nodes = [True, False]
node_size = [2, 4, 8, 16, 32, 64, 128, 256, 512]
dropout_rate = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5]
epochs = [1, 2, 3, 4, 5, 10, 20, 50, 100]
learning_rate = [0.0001, 0.001, 0.01, 0.1]
batch_size = [1, 2, 4, 8, 16, 32, 64, 128]

In [ ]:
def input_layer(nodes: int, activation_function: str):
    """
    Creates an input layer for a neural network.

    Parameters:
    nodes (int): Number of neurons in the input layer.
    activation_function (str): Activation function to be used in the input layer.

    Returns:
    Dense: A Keras Dense layer with the specified number of nodes and activation function.
    """
    return Dense(nodes, activation=activation_function, input_shape=(1,))


def hidden_layer(nodes: int, activation_function: str):
    """
    Creates a hidden layer for a neural network.

    Parameters:
    nodes (int): The number of neurons in the hidden layer.
    activation_function (str): The activation function to be applied to the layer.

    Returns:
    Dense: A Keras Dense layer with the specified number of neurons and activation function.

    Example:
    >>> layer = hidden_layer(10, 'relu')
    >>> print(layer)
    <keras.src.layers.core.dense.Dense object at 0x...>
    """
    return Dense(nodes, activation=activation_function)


def output_layer():
    """
    Creates an output layer for a neural network.

    Returns:
    Dense: A Keras Dense layer with one neuron and a linear activation function.
    """
    return Dense(1, activation='linear')


def create_hidden_layers(model, num_layers: int, start_nodes: int, reduce_nodes: bool, activation_function: str, dropout_rate: float):
    """
    Adds hidden layers to a model with the option to either reduce or keep constant the number of neurons.

    Parameters:
    - model (Sequential): The Keras Sequential model to which the layers will be added.
    - num_layers (int): Number of hidden layers to add.
    - start_nodes (int): The number of neurons in the first hidden layer.
    - reduce_nodes (bool): If True, the number of neurons will decrease in each layer.
    - activation_function (str): The activation function to use for the hidden layers.
    
    Returns:
    - model (Sequential): The updated Keras Sequential model with added hidden layers.
    """
    
    for i in range(num_layers):
        if reduce_nodes:
            # Decrease the number of neurons by half in each layer
            nodes = max(1, int(start_nodes / (2 ** i)))  # Avoid going below 1 neuron
        else:
            # Keep the number of neurons constant
            nodes = start_nodes

        model.add(hidden_layer(nodes, activation_function))
        model.add(Dropout(dropout_rate))  # Add dropout layer
        # Add a dropout layer after each hidden layer

    return model


def build_model(input_nodes: int, input_activation: str,
                num_hidden_layers: int, hidden_nodes: int, reduce_nodes: bool = True, hidden_activation: str = 'ReLU', dropout_rate: float = 0.0,
                optimizer: str = 'adam', learning_rate: float = 0.01, batch_size: int = 32):
    
    """
    Builds a customizable neural network model using the Sequential API.

    This function creates a neural network model with a specified number of hidden layers, 
    customizable hidden node sizes (with an option for reducing nodes), activation functions, 
    and dropout regularization.

    Parameters:
    ----------
    input_nodes : int
        The number of nodes in the input layer.
    
    input_activation : str
        The activation function for the input layer (e.g., 'ReLU', 'sigmoid', 'tanh').
    
    num_hidden_layers : int
        The number of hidden layers to be added to the model.
    
    hidden_nodes : int
        The number of nodes in the first hidden layer. The number of nodes in subsequent 
        layers will decrease if `reduce_nodes` is set to True.
    
    reduce_nodes : bool, default=True
        Whether to reduce the number of nodes in each subsequent hidden layer. If False, 
        all hidden layers will have the same number of nodes as the first hidden layer.
    
    hidden_activation : str, default='ReLU'
        The activation function for all hidden layers (e.g., 'ReLU', 'sigmoid', 'tanh').
    
    dropout_rate : float, default=0.0
        The dropout rate applied to the hidden layers to prevent overfitting. A value between 0 and 1.

    Returns:
    -------
    model : keras.Sequential
        A compiled Sequential model ready to be trained.
    """

    model = Sequential()
    # Add the input layer
    model.add(input_layer(input_nodes, input_activation))
    # Add hidden layers
    model = create_hidden_layers(model, num_hidden_layers, hidden_nodes, reduce_nodes, hidden_activation, dropout_rate)
    # Add the output layer
    model.add(output_layer())

    # Choose optimizer based on input
    if optimizer == 'adam':
        opt = Adam(learning_rate=learning_rate)
    elif optimizer == 'sgd':
        opt = SGD(learning_rate=learning_rate)
    elif optimizer == 'rmsprop':
        opt = RMSprop(learning_rate=learning_rate)
    return model

In [ ]:
# Compile the model with the Adam optimizer, and custom learning rate
optimizer = Adam(learning_rate=0.01)  # You can change the learning rate as needed

model.compile(optimizer=optimizer, loss='mean_squared_error')

# Train the model with custom batch size
model.fit(X, y, epochs=100, batch_size=2, verbose=1)  # Using batch size of 2

# Predict with the trained model
predictions = model.predict(X)

# Print the predictions
print("Predictions: ", predictions.flatten())

In [ ]:
l = widgets.Text(value='                                       Machine Learning Overfit/Generalization Demo, Prof. Michael Pyrcz and John Eric McCarthy II, The University of Texas at Austin',
                 layout=Layout(width='950px', height='30px'))

n = widgets.IntSlider(min=15, max = 80, value=30, step = 1, description = 'n',orientation='horizontal', style = {'description_width': 'initial'}, continuous_update=False)
split = widgets.FloatSlider(min=0.05, max = .95, value=0.20, step = 0.05, description = 'Test %',orientation='horizontal',style = {'description_width': 'initial'}, continuous_update=False)
std = widgets.FloatSlider(min=0, max = 50, value=0, step = 1.0, description = 'Noise StDev',orientation='horizontal',style = {'description_width': 'initial'}, continuous_update=False)
degree = widgets.IntSlider(min=1, max = 12, value=1, step = 1, description = 'Model Order',orientation='horizontal', style = {'description_width': 'initial'}, continuous_update=False)

ui = widgets.HBox([n,split,std,degree],)
ui2 = widgets.VBox([l,ui],)